In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np

train = pd.read_csv("chennai_reviews.csv")
train["Review_Text"] = train["Review_Text"].fillna("the")
train["Sentiment"] = pd.to_numeric(train["Sentiment"], errors='coerse')
train["Sentiment"] = train["Sentiment"].fillna(2)

nltk.download('stopwords')

def convert_to_words(raw_text):
    letters_only = re.sub("[^a-zA-Z]", 
                      " ",                                         
                      str(raw_text))
    words = letters_only.lower().split()
    stop_words = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stop_words]
    return " ".join(meaningful_words)
    
clean_review = convert_to_words(train["Review_Text"][0])

num_reviews = train["Review_Text"].size

clean_train_reviews = []

for i in range(0, num_reviews):
    clean_train_reviews.append(convert_to_words(train["Review_Text"][i]))

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

np.asarray(train_data_features)

forest = RandomForestClassifier(n_estimators = 100)

forest = forest.fit(train_data_features, train["Sentiment"])

test = pd.read_csv("7282_1.csv")

num_reviews1 = test["reviews.text"].size

clean_test_reviews = []
            
for i in range(0, 25):
    clean_review = convert_to_words(test["reviews.text"][i])
    clean_test_reviews.append(clean_review)
    
full_reviews = [x for x in clean_test_reviews if x != ""]
        
english_reviews = [x for x in full_reviews if detect(x) == "en"]        
            
test_data_features = vectorizer.transform(english_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

output = pd.DataFrame(data={"sentiment":result})

head = output.head(15)
 
print(head)

[nltk_data] Downloading package stopwords to /home/andrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
    sentiment
0         3.0
1         3.0
2         3.0
3         3.0
4         3.0
5         3.0
6         3.0
7         3.0
8         3.0
9         1.0
10        3.0
11        3.0
